# Homework2
Roy Adams -- CS 352 Spring 2019

### Problem 1

In [2]:
spam_corpus = [["i", "am", "spam", "spam", "i", "am"], ["i", "do", "not", "like", "that", "spamiam"]]
ham_corpus = [["do", "i", "like", "green", "eggs", "and", "ham"], ["i", "do"]]
combined_corpus = ["i", "am", "spam", "spam", "i", "am", "i", "do", "not", "like", "that", "spamiam", "do", "i",
                    "like", "green", "eggs", "and", "ham", "i", "do"]
combined_corpus = list(dict.fromkeys(combined_corpus))

hash_spam = {}
for i in spam_corpus:
    for j in i:
        if j not in hash_spam:
            hash_spam[j] = 1
        else:
            hash_spam[j] += 1

hash_ham = {}
for i in ham_corpus:
    for j in i:
        if j not in hash_ham:
            hash_ham[j] = 1
        else:
            hash_ham[j] += 1

hash_prob = {}
for i in combined_corpus:
    g = 0;
    b = 0;
    if i in hash_ham:
        g = 2 * hash_ham[i]
    if i in hash_spam:
        b = hash_spam[i]
    if g + b >= 1:
        hash_prob[i] = max(0.1,
                           min(.99,
                               min(1.0, b / len(spam_corpus)) /
                               (min(1.0, b / len(spam_corpus)) + min(1.0, g / len(ham_corpus)))))
print("Hash Table of Spam/Ham Corpus: ")
for i in hash_prob:
    print(i, hash_prob[i])

Hash Table of Spam/Ham Corpus: 
like 0.3333333333333333
ham 0.1
spamiam 0.99
am 0.99
i 0.5
green 0.1
that 0.99
and 0.1
not 0.99
spam 0.99
eggs 0.1
do 0.3333333333333333


In [9]:
# Test spam e-mail
test_spam = ["i", "like", "spam", "i", "do", "not", "like", "ham"]
# Test ham e-mail
test_ham = ["i", "like", "ham", "i", "do", "not", "love", "eggs", "though"]


def spam_test(message):
    prod = 1
    prod_complement = 1
    for i in message:
        if i in hash_prob:
            prod *= hash_prob[i]
            prod_complement *= 1 - hash_prob[i]
        eprint("Spam:", spam_test(test_spam))lse:
            prod *= 0.4
            prod_complement *= 0.6

    prob = prod / (prod + prod_complement)
    return prob

In [10]:
print("Spam:", spam_test(test_spam))

Spam: 0.9927073837739289


In [11]:
print("Ham:", spam_test(test_ham))

Ham: 0.11956521739130425


### Problem 2

In [12]:
from probability import BayesNet, enumeration_ask, elimination_ask

# Utility variables
T, F = True, False

wet_grass = BayesNet([
    ('Cloudy', '', 0.5),
    ('Sprinkler', 'Cloudy', {T: 0.1, F: 0.5}),
    ('Rain', 'Cloudy', {T: 0.8, F: 0.2}),
    ('WetGrass', 'Sprinkler Rain', {(T, T): 0.99, (T, F): 0.9, (F, T): 0.9, (F, F): 0.0}),
    ])


The total number of independent values in this situation is 16 since there are four variables each with possible T/F values. 2^4 = 16.

The number of independent values for a Bayesian network in this domain is 9. 1 for Cloudy, 2 for both Rainy and Sprinkler, and 4 for WetGrass.

In [13]:
print(enumeration_ask('Cloudy', dict(), wet_grass).show_approx())

False: 0.5, True: 0.5


$P(Cloudy) = <0.5, 0.5>$ This is directly from graph.


In [14]:
print(enumeration_ask('Sprinkler', dict(Cloudy=T), wet_grass).show_approx())

False: 0.9, True: 0.1


$P(Sprinkler|Cloudy) = <0.1, 0.9>$ This is directly from graph.


In [15]:
print(enumeration_ask('Cloudy', dict(Sprinkler=T, Rain=F), wet_grass).show_approx())

False: 0.952, True: 0.0476


$P(Cloudy|Sprinkler, -Rain)$  
$= a \cdot P(Sprinkler, -Rain|Cloudy) \cdot P(Cloudy)$  
$= a \cdot <0.1 \cdot 0.2 \cdot 0.5, 0.5 \cdot 0.8 \cdot 0.5> = <0.0476, 0.9524>$

In [16]:
print(enumeration_ask('WetGrass', dict(Cloudy=T, Sprinkler=T, Rain=T), wet_grass).show_approx())

False: 0.01, True: 0.99


$P(WetGrass|Cloudy, Sprinkler, Rain) = <0.99, 0.01>$ This is directly from graph.

In [17]:
print(enumeration_ask('Cloudy', dict(WetGrass=F), wet_grass).show_approx())

False: 0.639, True: 0.361


$P(Cloudy|-WetGrass) = a * P(-WetGrass|Cloudy) * P(Cloudy)$  
$ = a * [P(-WetGrass|Sprinkler, Rain) * P(Sprinkler|Cloudy) * P(Rain|Cloudy)$  
$\> + P(-WetGrass|Sprinkler, -Rain) * P(Sprinkler|Cloudy) * P(-Rain|Cloudy)$  
$\> + P(-WetGrass|-Sprinkler, Rain) * P(-Sprinkler|Cloudy) * P(Rain|Cloudy)$  
$\> + P(-WetGrass|-Sprinkler, -Rain) * P(-Sprinkler|Cloudy) * P(-Rain|Cloudy)] * P(Cloudy)$  
$ = a * <[0.01 * 0.1 * 0.8 + 0.9 * 0.1 * 0.2 + 0.9 * 0.9 * 0.8 + 0.0 * 0.9 * 0.2] * 0.5,$  
$<[0.01 * 0.5 * 0.2 + 0.9 * 0.5 * 0.8 + 0.9 * 0.5 * 0.2 + 0.0 * 0.5 * 0.8] * 0.5> = <0.639, 0.361>$